In [0]:
# installing necessery packages
!pip install lyricsgenius
!pip install -U pip
!pip install -U dill
!pip install nltk==3.5b1

In [0]:
# some imports
import lyricsgenius
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import tokenize
from nltk.tokenize import LineTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
import random
from nltk.lm.preprocessing import flatten
from nltk.lm import Vocabulary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Foreword**

As a first iteration of an exercise I downloaded the lyrics of all the songs of Oxxymiron (88 songs) and built uni- and bigram model training them on the songs. Results were pretty satisfying but I had problems with culculating perplexity of the models. 

Following your advices I updated nltk to v3.5b1 and the problem with perplexity has gone. 

I also increased corpus size by adding all the songs of Slava KPSS, Noize MC and FACE (257, 271 and 135 songs, respectively). The final corpus consisted of approximately 40k lines of rap. However, when generating the lyrics with models trained on this corpus I faced anoother problem: computational. The lyrics were generating too slowly (for not MLE models): for example, I left the model generating the whole night (at least 8 hours) and in the morning it only generated **TWO** lines (10 words long each). I'm not even mentioning perplexities, I think it could take ages to compute them.

Thus, I decided to decrease the corpus a little bit to compare models and to show that they work, and left 20 songs of each artist in order to maintain style differentiation :) 

(to download the whole corpora just set the parameter "max_songs" in search_artist method to None)



# **Collecting and prepocessing the data**

In [0]:
# initializing genius class
genius = lyricsgenius.Genius("iihP9u6cVKE79Xtu99OH6rVZlMUzIMeOvIVs8BDQNDeVmZhXJXEaI28E6QOc4BdB")
genius.remove_section_headers = True
genius.skip_non_songs = False 
genius.excluded_terms = ["(Remix)", "(Live)"]

In [0]:
# Collecting Oxxxymiron songs
Oxxxymiron = genius.search_artist("Oxxxymiron",max_songs=20, sort="title")

Searching for songs by Oxxxymiron...

Song 1: "16 Bars Acapella"
Song 2: "Afterparty (Demo)"
Song 3: "AI Ogon"
Song 4: "Amplify and simplify (Freestyle)"
Song 5: "CCTV"
Song 6: "Darkside"
Song 7: "Freestyle #1"
Song 8: "Ganz Promo Tune"
Song 9: "Hangover"
Song 10: "HPL"
Song 11: "IMPERIVM"
Song 12: "Intro"
Song 13: "OXXXYMIRON"
Song 14: "Russky Cockney"
Song 15: "Shade 45 Freestyle (Идея)"
Song 16: "Street Freestyle battle"
Song 17: "Ultima Thule"
Song 18: "Unreleased Track"
Song 19: "Unreleased Track 2"
Song 20: "Unreleased Track 3"

Reached user-specified song limit (20).
Done. Found 20 songs.


In [0]:
# Collecting Slava KPSS songs
SlavaKPSS = genius.search_artist("Слава КПСС",max_songs=20, sort="title")

Searching for songs by Слава КПСС...

Changing artist name to 'Слава КПСС (Slava KPSS)'
Song 1: "0лик (Zero)"
Song 2: "100 barz (Нахуй всех!)"
Song 3: "12 мая (12 may)"
Song 4: "16 проблем (InDaBattle 3 Round 2)"
Song 5: "18 марта (18th of March)"
Song 6: "2,5 человека (DSV Battle Round 1)"
Song 7: "30 новых авто (30 New Cars)"
Song 8: "30 новых авто (30 new cars) [Alternative version]"
Song 9: "5500"
Song 10: "Black Stalin"
Song 11: "BLUNTCUT"
Song 12: "Burlit"
Song 13: "Comeback"
Song 14: "CUCKOLDINI"
Song 15: "Donbass"
Song 16: "FACE DISS (FACEFVCK)"
Song 17: "Grime Thing"
Song 18: "KIBERDEMON FREESTULE"
Song 19: "King Ring (Freestyle)"
Song 20: "MONEY TALK"

Reached user-specified song limit (20).
Done. Found 20 songs.


In [0]:
# Collecting NoizeMC songs
NoizeMC = genius.search_artist("Noize MC", max_songs=20, sort="title")

Searching for songs by Noize MC...

Changing artist name to 'Noize MC'
Song 1: "+−0"
Song 2: "10 суток (Сталинград) (10 days)"
Song 3: "12 обезьян (Ляпис Трубецкой cover)"
Song 4: "18:30"
Song 5: "200+"
Song 6: "220"
Song 7: "2 хороших девочки (2 good girls)"
Song 8: "3П (Правдивая песня пиздобола) (3P (Liar’s true song)"
Song 9: "3П (Правдивая песня пиздобола) [Фристайл] (3P (Liar’s true song) [Freestyle])"
Song 10: "5П (Песня похуиста про получение пиздюлей) (5P)"
Song 11: "Chasing the Horizon"
Song 12: "Come $ome All (Тоталитарный трэпъ) (Come Some All)"
Song 13: "Ctrl+Alt+Delete"
Song 14: "Face a la Mer (Faced the sea)"
Song 15: "Funk Soul Brother (Фристайл) (Freestyle)"
Song 16: "Game Over"
Song 17: "Hard Reboot"
Song 18: "Heart Shaped Box + Фристайл (Freestyle)"
Song 19: "I Am the Rain"
Song 20: "I’ll Be Back (Фристайл) (Freestyle)"

Reached user-specified song limit (20).
Done. Found 20 songs.


In [0]:
# Collecting FACE songs
FACE = genius.search_artist("FACE",max_songs=20,  sort="title")

Searching for songs by FACE...

Song 1: "162"
Song 2: "24 на 7"
Song 3: "All Good"
Song 4: "APRIL  8"
Song 5: "Baby"
Song 6: "Baby Face"
Song 7: "Blazer"
Song 8: "Enter The Void"
Song 9: "Forever Young"
Song 10: "Holiday"
Song 11: "Ian Connor"
Song 12: "I dropped Gucci"
Song 13: "I Hate Humans (Unreleased)"
Song 14: "Instagram"
Song 15: "I Want"
Song 16: "January"
Song 17: "Kanji"
Song 18: "Karamel"
Song 19: "KILL"
Song 20: "MASK"

Reached user-specified song limit (20).
Done. Found 20 songs.


In [0]:
# saving the lyrics of all songs to separate files
Oxxxymiron.save_lyrics(extension='txt')
SlavaKPSS.save_lyrics(extension='txt')
NoizeMC.save_lyrics(extension='txt')
FACE.save_lyrics(extension='txt')

Wrote `Lyrics_Oxxxymiron.txt`
Wrote `Lyrics_СлаваКПССSlavaKPSS.txt`
Wrote `Lyrics_NoizeMC.txt`
Wrote `Lyrics_FACE.txt`


In [0]:
# opening them back 
Oxy_lyrics = open("Lyrics_Oxxxymiron.txt","r")
Slava_lyrics = open("Lyrics_СлаваКПССSlavaKPSS.txt","r")
Noize_lyrics = open("Lyrics_NoizeMC.txt","r")
FACE_lyrics = open("Lyrics_FACE.txt","r")

In [0]:
# tokenizing the lyrics, lowering all the words (we will capitalize them back soon :) and leaving all punkt as it is (it will make our generated lyrics
# look more natural)
# In the end we get list of lists (lines in songs) of tokenized words, 
# so Oxy_tokenized[0] is not the 1st song of Oxxy but the 1st line of all the downloaded Oxy lyrics 
Oxy_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Oxy_lyrics]
Slava_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Slava_lyrics]
Noize_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Noize_lyrics]
FACE_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in FACE_lyrics]  

In [0]:
# checking the length of the corpora (num of lines of rap)
print(len(Oxy_tokenized),len(Slava_tokenized),len(Noize_tokenized),len(FACE_tokenized))

1157 1229 1222 765


In [0]:
# combine all the lyrics into one list
all_lyrics = []
all_lyrics.extend(Oxy_tokenized)
all_lyrics.extend(Slava_tokenized)
all_lyrics.extend(Noize_tokenized)
all_lyrics.extend(FACE_tokenized)
print(len(all_lyrics))
print(len(Oxy_tokenized)+len(Slava_tokenized)+len(Noize_tokenized)+len(FACE_tokenized))

4373
4373


I also had an idea of first of all dividing the lyrics into sentences and leave \n synmbols as indicators of new line, so the generator could make newline itself. But the artists didnt include full stop marks to their Lyrics, so itis impossible to tokenize sentences.

In [0]:
# Firstly, I was thinking, that adding \n symbol is a good idea to let the model know, where to start the next line,
# but then i neglected the idea
# for strokes in tokenized_text:
#   strokes.append('\n')

Then I defined a function the generate a rap lyrics given model, line size and number of lines. I also included into this function detokenizer and capitalizer to make the lyrics look pretty.

In [0]:
# def a function to generate rap text given a model
def rap_gen(model, num_lines = 20, num_words_in_line = 20, random_seed = 0):
  """
  The function generates rap lyrics. This is a very creative process, so be aware,
  that the number of words in lines and even number of lines may be less than you desired 
  (it totally depends on the function's inspiration (and random_seed)) :)
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_lines: Max no. of lines to generate.
    :param num_words_in_line: Max no. of words in line to generate.
    :param random_seed: Seed value for random.
    """
  detokenize = TreebankWordDetokenizer().detokenize
  for i in range(num_lines):
    content = []
    for token in model.generate(num_words_in_line, random_seed=random_seed+i):
      if token == '<s>':
        continue
      if token == '</s>':
        break
      content.append(token)
    content = detokenize(content)
    if bool(content):
      i = 0
      while not content[i].isalpha():
          i += 1
          if i > len(content)-1:
            break
      print(content[i:].capitalize())



In [0]:
# divide into test and train set. Leave test size not very big to better train the model and faster estimate perplexity. 
train, test = train_test_split(all_lyrics, test_size=0.2, random_state=11)

# **Boulding models**
I decided to build all in all 9 models: MLE, Laplace smoothing and KneseNey smoothing interpolation based on uni-, bi- and trigrams.

After each model there is an example of generated rap.

BE AWARE, RUSSIAN CURSE AND SWEAR WORDS ARE NOT CENSORED. 18+ CONTENT. 

If you are not Okay with it, go back and change the list of artists to, for example, Yurii Loza, Sofia Rotaru or Valeriy Meladze.

***Unigram* models**

In [0]:
model1_MLE = MLE(1)
train_data1_1, padded_sents1_1 = padded_everygram_pipeline(1, train)
model1_MLE.fit(train_data1_1, padded_sents1_1)

In [0]:
rap_gen(model1_MLE, num_lines=10, num_words_in_line=10,random_seed = 12)

Каждый о оксибутиратом?! дебила бы среди перевороты нас
Бля папки пальцы тлеющий long «``you про.
По ноль этим будут бить про о в памяти
Я! прожигает check ёбнем 'em у отряд тоталитарный ”
Гроб как за и её о бля неуловимой! в
Лимонова сотрясается я в с по-твоему обсираешь ,) для
It обсуди всем radmir кем когда как иду брат бесплатный
От сердце ли кто-то другом — в about более бодрый
Хочу парней с хочешь боба ни хочу ты мы funk
Face пепла неуловимый как told скоро спасибо куда-то который а


In [0]:
model1_Lapl = Laplace(1)
train_data1_2, padded_sents1_2 = padded_everygram_pipeline(1, train)
model1_Lapl.fit(train_data1_2, padded_sents1_2)

In [0]:
rap_gen(model1_Lapl, num_lines=10,num_words_in_line=10,random_seed = 12)

Кодати от палева good! дыр вам сосут поближе не
В плачу плавает тебя think бицевском hype бейп протеста cross
Полет они чёрных в бычки против от вряд плаву
Я' прошу made ядерный (ты пиздуй то ”
Длинном контингент золото истиной звать от в нужна! время
Мгновенно собой эфире вкладчики с полу отличный ;) есть
Take отличный горы you крикни кто-то конечно как в было
Песни серию масштабах любимую жить — виски i в в
Фьорд пломбы с фристайлить в ныть фрэнд тут наблюдаешь prophets
Of по нужна кому-то антихайп скитальца создал любовь либерию более


In [0]:
model1_KNI = KneserNeyInterpolated(1)
train_data1_3, padded_sents1_3 = padded_everygram_pipeline(1, train)
model1_KNI.fit(train_data1_3, padded_sents1_3)

In [0]:
rap_gen(model1_KNI, num_lines=10,num_words_in_line=10,random_seed = 12)

Меж поздравляю получал блока asbestos какого-то дерьма смешно представляю очках
Давидофф почетное похожий строки виллу герасима более-менее выше рай безумно
Автэ приходят подписывал хайпить демо-тейп грязь рай пойду ебет похотливый
Честная b раскисли брэшвилл экс-ан-прованс bredrin тнт пот счастью ”
Имею мечты кости лечь конч позер давать пластоу and ебанись
Наскучило слушать часам домосед самом проанализировал покрываюсь алкоголя die качать
Взгляни покрываюсь запомните возраста мо мокрая метро любовью двигаем грузин
Посвящаю серых народ надевал клоны яркие домой больного дальнейшей дай-дай
Убьем пою самоубийцы убил дадут плачут убитый телек огромный ваш
Бьёшь предельно пластоу метастазами выбор скажи слюни надеюсь мэйна глупо


Model based on unigrams generate a list of unconnected words, and it can not be called "nornal lyrics for rap song".

**Everygram models (until *bigram*)**

In [0]:
model2_MLE = MLE(2)
train_data2_1, padded_sents2_1 = padded_everygram_pipeline(2, train)
model2_MLE.fit(train_data2_1, padded_sents2_1)

In [0]:
rap_gen(model2_MLE, num_lines=10, num_words_in_line=10,random_seed = 12)

Все под оксибутиратом
Приводит иногда я гнал
Лет десяток!
Я, сын, я - это пидоролитет
Me?
Достоин даже цикад
На твой половой зрелости аттестат?
У нас не такой, посмотрите-ка, что опопсел?


In [0]:
model2_Lapl = Laplace(2)
train_data2_2, padded_sents2_2 = padded_everygram_pipeline(2, train)
model2_Lapl.fit(train_data2_2, padded_sents2_2)

In [0]:
rap_gen(model2_Lapl, num_lines=10, num_words_in_line=10,random_seed = 12)

Для предупреждения мы двигаем всех, если ты пёрнул в
Потому что у
Этой бригады мои бродяги, but your lens
Берегись тут мафия носит скорее погоны мвд
И страх, ждать шестьдесят дней вот более-менее всё ещё
По крайней мере
Неизбежен: « джианни мора »
У нас не твой друг и умереть молодым


In [0]:
model2_KNI = KneserNeyInterpolated(2)
train_data2_3, padded_sents2_3 = padded_everygram_pipeline(2, train)
model2_KNI.fit(train_data2_3, padded_sents2_3)

In [0]:
rap_gen(model2_KNI, num_lines=10, num_words_in_line=10,random_seed = 12)

Медсестра
Давать мне от того, давай, да? )
Авторы задрочены
Честная, так . я - это пидоролитет
Иметь со мной
Насколько я читаю какие-то странные цацы
Взгляд – обожаю
Посаженых мойрой, не изобретёнными
Убьем эту тупизну рогами пробьет
Бьют тебя нихуя


Model based on bigrams generate quite good texts. Manually corrected, generated text can even be used as lyrics to some rap song (yes, I'm not a rap expert).

**Everygram models (until *trigram*)**

In [0]:
model3_MLE = MLE(3)
train_data3_1, padded_sents3_1 = padded_everygram_pipeline(3, train)
model3_MLE.fit(train_data3_1, padded_sents3_1)

In [0]:
rap_gen(model3_MLE, num_lines=10, num_words_in_line=10,random_seed = 12)

Money, money, blud . gimme some money
Это big up
Каждую минуту и секунду, каждый день
Больше четверти века набекрень голова
Коню за продакшн, зов кандалакши
Только польщён


In [0]:
model3_Lapl = Laplace(3)
train_data3_2, padded_sents3_2 = padded_everygram_pipeline(3, train)
model3_Lapl.fit(train_data3_2, padded_sents3_2)

In [0]:
rap_gen(model3_Lapl, num_lines=10, num_words_in_line=10,random_seed = 12)

Бык, но вздрогнул с немцовских пуль
Крикни « ауе », потом второй, и
Эта наколка
Download mixtape
Вырви и выброси . серотонин в минусе
Мной сука цвета, будто я герой сартра
Трачу свои деньги — и с наганом


In [0]:
model3_KNI = KneserNeyInterpolated(3)
train_data3_3, padded_sents3_3 = padded_everygram_pipeline(3, train)
model3_KNI.fit(train_data3_3, padded_sents3_3)

In [0]:
rap_gen(model3_KNI, num_lines=10, num_words_in_line=10,random_seed = 12)

Медсестра
Давать мне, сука, мне в директ
Авторы задрочены
Честная, как в спортлото
Иметь со словом
Насколько я старый . стал троцкистом
Взгляд – это капец экстаз
Посаженых мойрой, следите
Убьем эту падлу? )
Бьют тебя, сука, что хотел


Model based on bigrams mostly generate copies of lines of particular songs. 

# **Model evaluation**

Can you also give some advise on a model evaluation with perplexity. Thing I did below, unfortunately, do not give desired result.

In [0]:
test_data1,_ = padded_everygram_pipeline(1, test)
test_data2,_ = padded_everygram_pipeline(2, test)
test_data3,_ = padded_everygram_pipeline(3, test)

In [0]:
test_gen1 = []
test_gen2 = []
test_gen3 = []
for line in test_data1:
  test_gen1.extend(line)
for line in test_data2:
  test_gen2.extend(line)
for line in test_data3:
  test_gen3.extend(line)

In [0]:
print('Perplexity for unigram MLE model is: {}'.format(model1_MLE.perplexity(test_gen1)))

Perplexity for unigram MLE model is: inf


In [0]:
print('Perplexity for unigram Laplace model is: {}'.format(model1_Lapl.perplexity(test_gen1)))

Perplexity for unigram Laplace model is: 1454.318239940892


In [0]:
print('Perplexity for unigram KneserNeyInterpolated model is: {}'.format(model1_KNI.perplexity(test_gen1)))

Perplexity for unigram KneserNeyInterpolated model is: 7222.999999998707


In [0]:
print('Perplexity for bigram MLE model is: {}'.format(model2_MLE.perplexity(test_gen2)))

Perplexity for bigram MLE model is: inf


In [0]:
print('Perplexity for bigram Laplace model is: {}'.format(model2_Lapl.perplexity(test_gen2)))

Perplexity for bigram Laplace model is: 1186.4936058181775


In [0]:
print('Perplexity for bigram KneserNeyInterpolated model is: {}'.format(model2_KNI.perplexity(test_gen2)))

Perplexity for bigram KneserNeyInterpolated model is: 2820.7160107988434


In [0]:
print('Perplexity for trigram MLE model is: {}'.format(model3_MLE.perplexity(test_gen3)))

Perplexity for trigram MLE model is: inf


In [0]:
print('Perplexity for trigram Laplace model is: {}'.format(model3_Lapl.perplexity(test_gen3)))

Perplexity for trigram Laplace model is: 795.3070857094062


In [0]:
print('Perplexity for trigram KneserNeyInterpolated model is: {}'.format(model3_KNI.perplexity(test_gen3)))

Perplexity for trigram KneserNeyInterpolated model is: 1115.0942101808953


**Comment on n-gram models.**

According to perplexity, trigram Laplace model is better than any other. But still, perplexity is quite high (at least comparing with numbers from Dan Jurafsky book). 
I find at least two reasons for that: 

1) The training sample is too small (computational boundaries)

2) Russian rap is not very suitable for n-gram model: I believe, that in russian rap lyrics there are a lot of unique word collocations, which you can not meet in real russian language or in any other rap song. That is why in generated lyrics there are a lot of direct citations from songs: there are no other possible variants to make such a phrase. 

In order to somehow overcome the second obstacle I decided to experiment with character by character generator (see section beloow).

# **Experiments with RNN**

> As an experiment I wanted to play with a simple RNN. I found tutorial on implementing RNN for text generation task from Tensorflow ([link](https://www.tensorflow.org/tutorials/text/text_generation)). It was about generating text character by character, and I was interested in how it's gonna was for rap lyrics generation task. Most lines of code are directly borrowed from there with just little changes. Below you can see results of an experiment. Since the RNN generated results much faster, I was able to experiment with corpora of all songs of 4 rappers.



In [0]:
import tensorflow as tf

import numpy as np
import os
import time

In [0]:
# download lyrics of all songs of Oxxxymiron, Slava KPSS, Noize MC and FACE. 
Oxy_lyrics2 = open('Lyrics_Oxxxymiron.txt','rb').read().decode(encoding='utf-8')
Slava_lyrics2 = open("Lyrics_СлаваКПССSlavaKPSS.txt",'rb').read().decode(encoding='utf-8')
Noize_lyrics2 = open("Lyrics_NoizeMC.txt",'rb').read().decode(encoding='utf-8')
FACE_lyrics2 = open("Lyrics_FACE.txt",'rb').read().decode(encoding='utf-8')

In [3]:
# concatenate to make the final corpus
# check the length of corpora
text = Oxy_lyrics2 + '\n' + Slava_lyrics2 + '\n' + Noize_lyrics2 + '\n' + FACE_lyrics2
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1453063 characters


In [4]:
# Let's look at the si`e of vocabulary
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

191 unique characters


Why so many symbols? Let's take a look at the exact symbols.

In [5]:
print(vocab)

['\t', '\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\xa0', '«', '»', 'À', 'É', '×', 'à', 'á', 'ç', 'è', 'é', 'ê', 'ë', 'û', '́', 'Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'є', 'і', 'ї', 'ғ', 'Қ', 'қ', 'ң', 'ұ', '\u2005', '\u200a', '\u200e', '–', '—', '‘', '’', '“', '”', '„', '…', '\u205f', '⍰']


What are those '\u2005', '\u200a', et? After quick check I found out that these are just spaces decoded incorrectly.

Let's quickly turn them back to spaces. 

Also there are symbols from Latin and Kazakh languages, since there are lines in French, Spanish, English and Kazakh in some songs in corpora. I decided leave them there: the more languages, the more interesting! 

In [0]:
char_to_replace = ['\xa0','\u2005','\u200a','\u200e','\u205f','⍰']
for char in char_to_replace:
  text = text.replace(char,' ')

In [0]:
# recreate the vocabulary
vocab = []
vocab = sorted(set(text))

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=100

In [51]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
445/445 [==============================] - 15s 33ms/step - loss: 2.7855
Epoch 2/100
445/445 [==============================] - 15s 34ms/step - loss: 2.2855
Epoch 3/100
445/445 [==============================] - 15s 34ms/step - loss: 2.0656
Epoch 4/100
445/445 [==============================] - 15s 34ms/step - loss: 1.9119
Epoch 5/100
445/445 [==============================] - 15s 33ms/step - loss: 1.7918
Epoch 6/100
445/445 [==============================] - 15s 34ms/step - loss: 1.6874
Epoch 7/100
445/445 [==============================] - 15s 33ms/step - loss: 1.5901
Epoch 8/100
445/445 [==============================] - 15s 34ms/step - loss: 1.5019
Epoch 9/100
445/445 [==============================] - 15s 34ms/step - loss: 1.4211
Epoch 10/100
445/445 [==============================] - 15s 34ms/step - loss: 1.3507
Epoch 11/100
445/445 [==============================] - 15s 33ms/step - loss: 1.2910
Epoch 12/100
445/445 [==============================] - 15s 33ms/step - lo

In [52]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string, num_generate = 1000, temperature = 1.0):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = num_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [55]:
print(generate_text(model, start_string=u"Я люблю", num_generate = 1000))

Я люблю злость, переродо и опять даже твои губы на то, что может побыть непроводко

Жрать людям просто в голове, ведь умер 2Pac
Плак-плак-плак,ппак!


Это судайте, кого ных, ный лёд
Хотя бы не познали ключок бесконечный
Руслю у камня, ведь я не занят
Этот этот речь записа, по ногам
И что тебя анимпуказалось не нашли об этом

Глотая жопа, пацаны, как свет видит под подошвой
Стонет таем более толк, что не струселяем под мои глаза
И завшли не спасут
Чтоб умереть нужен стоит мок лучше видят громко?
Чтоб нас помню,я до диссионим, уловишь за сделан, не умирай
Покажи! (pow)
Islaw
Ты вы не первый и капкую и (рау-еф.)
Я клачевавная вся музыка — ксовать счетах
Я все еще увидеть конец циплер или яослалы
Твой стафф катрицию манек. Переперепериальные друли как кровавых сердцах

Шток держу в перехи, как Хавик
Пока (брр-а)
Вот, брошенных кровати
Канавтишься я когда-то жёг, я когда-то мог
На уши постмодерна
Турская
Пошёл на хуй посселан
Заслучаешься по гулям
И я бело всю нейт бетых
Все с утра...

Мое 

**Comment on RNN**

Well, this piece of lyrics at least does not directly cite any real song. So it is absolutely made up song. Main points about RNN:
1.   It is able to make new lines by itself
2.   It is able to generate word out of just a sequence of characters (fantastic)
3. Sometimes it generates nonexistent words :(
4. It needs more epochs of training, more layers in model, probably more training data, et.

Actually, the parameter, that is responsible for randomness in temperature. It is 1 by default, but lowering it, you can get less nonexistent words,  but more citations from songs.


In [59]:
print(generate_text(model, start_string=u"Он", num_generate = 1000, temperature = 0.65))

Они сходят за реклам
В мирове если рвфоле
Расправлю своих ветер разгонит в краску?
Вы, гра-ладим не раскрыл и «мелок», you a the case
Не проходит на мой хуй строги?
Вы самых не простить и сдавай на букевольку й пусто
Я сижу в темноте
И она не хуже в кармане есть этих строго похуй молитвы
Это правда
Если я убираю рассветь успел тебе против того что-то пидорас
Вот, б ты, красиво, без негосята беспредела вроде бы вы свили не снято нет Адрихаза — как в прошлом горядском молчами, что изменилось?
На борцом длянов — и я просто скучаешь
Брать воздух, как моя семья
Я хотел сказать, что в час миллионов Гуш
Понял то, что делал ты, стёкло очень окна, я придёт будет из колоса
Сегодня выйдут и потрихаю в своей заесчастней!

За совестных делах, она не кто только этого не хватало представляю крый уже о в храмоще с нами под Чемоден, она не нужна
Ха-а-ха, где тебя нет, где тебя нет
Ты ползёшь ниже плиндеребения
И я только всё, что было, не то ей в красках
Люди — кемная параша. Рок'н'ролл — это кал!

Пра

And finally I will share a masterpiece, created by bigram model, trained only on Oxxxymiron lyrics (unfortunately, I didn't save the code for that):



*Всегда видят лишь негатив, положив на весы*

*Коллеги твердят:``может хомяка положите туда*

*Abab — рифмовка никуда, ерунда!*

*Бра*

*Школу! сколько было всего, что я хочу сказать*

*И каждый вес мутит, а мой менталитет*

*Бит и бас, это наш год! до тебя*

*Как раньше: лёд всё тоньше*